In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
!pip install ortools

In [4]:
from ortools.sat.python import cp_model
import uuid
import random
import math
import itertools
import sys
# Generate a random UUID
unique_id = uuid.uuid4()


In [5]:
from transformers import BertTokenizer, BertTokenizerFast
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertModel
import torch.optim as optim
import json
import ast
import random
from torch import cuda
import re

import time

device = 'cuda' if cuda.is_available() else 'cpu'
print(device)


cuda


In [6]:
"""
0: {"constraint": "all_diff", "template":"templates_all_diff", "n_params":10},
    1: {"constraint": "<", "template":"templates_param1_strictely_lt_param2", "n_params":2},
    2: {"constraint": ">", "template":"templates_param1_strictely_gt_param2", "n_params":2},
    7: {"constraint": "==", "template":"templates_equal_constraint", "n_params":2},
    8: {"constraint": "!=", "template":"templates_different_than", "n_params":2},
    9: {"constraint": "<=", "template":"templates_less_than", "n_params":2},
    10: {"constraint": ">=", "template":"templates_greater_than", "n_params":2},
    12: {"constraint": "Sum", "template":"sum_params_equal_param", "n_params":10},
    13: {"constraint": "O_val_=", "template":"templates_equal_constraint", "n_params":2},
    14: {"constraint": "O_val_!=", "template":"templates_different_than", "n_params":2},
    15: {"constraint": "||=val", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    16: {"constraint": "||!=val", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3}
"""

# additional templates generted exclusively for simulating users during acquisition
# these templates were not used for generating used for finetuning BERT

templates_all_diff = [
    "Each element in {params} should be unique.",
    "Ensure uniqueness among the elements of {params}.",
    "All items in {params} must differ from each other.",
    "Ensure that each value in {params} stands alone.",
    "Distinctness is required for all elements in {params}.",
    "No two elements in {params} should be identical.",
    "Each of {params} should have a unique value.",
    "Avoid any duplication within {params}.",
    "There should be no common elements in {params}.",
    "Ensure that every element in {params} is distinct.",
    "Each entry in {params} must be individual and distinct.",
    "Guarantee that every one of {params} is unique.",
    "Maintain distinct values across {params}.",
    "No element should repeat within {params}.",
    "Every part of {params} must be exclusively its own.",
    "Insist on non-repetitive elements in {params}.",
    "Uphold uniqueness in each aspect of {params}.",
    "Enforce distinctiveness among {params}.",
    "Assure no overlaps in {params}.",
    "Demand individuality in each component of {params}."
]


sum_params_equal_param = [
    "Confirm that the combined total of {params} precisely matches {param}.",
    "Ascertain that the aggregate sum of {params} is exactly equal to {param}.",
    "Make sure that the total amount when {params} are summed up is {param}.",
    "Confirm that the combined total of {params} is {param}.",
    "Make sure the aggregate of {params} equals {param}.",
    "Check that the sum total of {params} amounts to {param}.",
    "Ascertain that {params}, when added, equal {param}.",
    "Ensure the cumulative amount of {params} reaches {param}.",
    "Guarantee that the total sum of {params} is precisely {param}.",
    "The total value of {params} should match {param}.",
    "Validate that the sum of {params} is consistent with {param}.",
    "Affirm that combining {params} totals to {param}.",
    "Be sure that the addition of {params} equals {param}.",
    "Ensure that {params}, summed up, result in {param}.",
    "Make certain the total calculation of {params} is {param}.",
    "The collective amount of {params} must tally to {param}.",
    "Confirm that the sum of all {params} equals {param}.",
    "The overall total of {params} should be exactly {param}.",
    "Verify the cumulative sum of {params} equates to {param}.",
    "Ascertain that the aggregate value of {params} is equal to {param}."
]


templates_param1_strictely_lt_param2 = [
    "{param1} is strictly inferior to {param2}.",
    "{param1} is definitively less than {param2}.",
    "{param1} is unambiguously smaller than {param2}.",
    "{param1} is categorically lower than {param2}.",
    "{param1} is explicitly beneath {param2}.",
    "{param1} is unmistakably below {param2}.",
    "{param1} is clearly under {param2}.",
    "{param1} is decidedly less than {param2}.",
    "{param1} is undeniably beneath {param2}.",
    "{param1} is absolutely lower than {param2}.",
    "{param1} is incontrovertibly smaller than {param2}.",
    "{param1} is inarguably less than {param2}.",
    "{param1} is undebatably beneath {param2}.",
    "{param1} is evidently below {param2}.",
    "{param1} is obviously under {param2}.",
    "{param1} is palpably less than {param2}.",
    "{param1} is manifestly lower than {param2}.",
    "{param1} is patently smaller than {param2}.",
    "{param1} is transparently beneath {param2}.",
    "{param1} is undisputedly below {param2}."
]

templates_param1_strictely_gt_param2 = [
    "{param1} is markedly higher than {param2}.",
    "{param1} towers above {param2}.",
    "{param1} stands out as greater than {param2}.",
    "{param1} holds a distinct advantage over {param2}.",
    "{param1} is considerably more than {param2}.",
    "{param1} surpasses {param2} significantly.",
    "{param1} is notably above {param2}.",
    "{param1} is profoundly greater than {param2}.",
    "{param1} is substantially higher than {param2}.",
    "{param1} exceeds {param2} clearly.",
    "{param1} is visibly more than {param2}.",
    "{param1} is outstandingly above {param2}.",
    "{param1} is demonstrably larger than {param2}.",
    "{param1} clearly outstrips {param2}.",
    "{param1} is significantly beyond {param2}.",
    "{param1} vastly outdoes {param2}.",
    "{param1} is majorly above {param2}.",
    "{param1} is emphatically greater than {param2}.",
    "{param1} is dominantly higher than {param2}.,"
    "{param1} is strictly superior to {param2}."
    ]

templates_different_than=[
    "{param1} varies from {param2}.",
    "{param1} and {param2} are not identical.",
    "{param1} diverges from {param2}.",
    "{param1} stands apart from {param2}.",
    "{param1} is distinct from {param2}.",
    "{param1} contrasts with {param2}.",
    "{param1} is not comparable to {param2}.",
    "{param1} holds no similarity to {param2}.",
    "{param1} is in contrast to {param2}.",
    "{param1} bears no resemblance to {param2}.",
    "{param1} and {param2} are dissimilar.",
    "{param1} does not mirror {param2}.",
    "{param1} is unlike {param2}.",
    "{param1} deviates from {param2}.",
    "{param1} and {param2} are in contrast.",
    "{param1} is not a match with {param2}.",
    "{param1} and {param2} do not align.",
    "{param1} is at variance with {param2}.",
    "{param1} diverges distinctly from {param2}.",
    "{param1} and {param2} are poles apart."
    ]

templates_less_than=[
    "{param1} falls below {param2}.",
    "{param1} is inferior to {param2}.",
    "{param1} ranks under {param2}.",
    "{param1} is beneath {param2}.",
    "{param1} is not as much as {param2}.",
    "{param1} trails {param2}.",
    "{param1} is lower than {param2}.",
    "{param1} doesn't reach the level of {param2}.",
    "{param1} is subordinate to {param2}.",
    "{param1} is behind {param2}.",
    "{param1} is under {param2}.",
    "{param1} should be smaller {param2}.",
    "{param1} is shorter than {param2}.",
    "{param1} is not as high as {param2}.",
    "{param1} is as important as {param2}.",
    "{param1} is secondary to {param2}.",
    "{param1} is outweighed by {param2}.",
    "{param1} is eclipsed by {param2}.",
    "{param1} is less significant than {param2}.",
    "{param1} does not surpass {param2}."
]


templates_greater_than= [
    "{param1} surpasses {param2}.",
    "{param1} is above {param2}.",
    "{param1} outstrips {param2}.",
    "{param1} is more than {param2}.",
    "{param1} exceeds the value of {param2}.",
    "{param1} is superior to {param2}.",
    "{param1} tops {param2}.",
    "{param1} is higher than {param2}.",
    "{param1} overshadows {param2}.",
    "{param1} outdoes {param2}.",
    "{param1} prevails over {param2}.",
    "{param1} is larger than {param2}.",
    "{param1} towers over {param2}.",
    "{param1} outnumbers {param2}.",
    "{param1} leads {param2}.",
    "{param1} dominates over {param2}.",
    "{param1} takes precedence over {param2}.",
    "{param1} outweighs {param2}.",
    "{param1} bests {param2}.",
    "{param1} has the edge over {param2}."
]

sum_params_equal_param= [
    "Confirm that the collective amount of {params} is {param}.",
    "Check that the combined sum of {params} equals {param}.",
    "Ascertain that the aggregate of {params} reaches {param}.",
    "Make sure the total calculation of {params} amounts to {param}.",
    "Ensure that the cumulative value of {params} matches {param}.",
    "Validate that the sum total of {params} is equivalent to {param}.",
    "Guarantee that the overall total of {params} is equal to {param}.",
    "Be certain that adding up {params} yields {param}.",
    "Affirm that the total of {params}, when added, equals {param}.",
    "Establish that the sum of all {params} is exactly {param}.",
    "Ensure the complete sum of {params} tallies up to {param}.",
    "Double-check that the aggregate sum of {params} is {param}.",
    "See to it that the total of {params} equates to {param}.",
    "Confirm that the entirety of {params} sums up to {param}.",
    "Insist that the cumulative effect of {params} totals {param}.",
    "Make certain that the overall effect of {params} is {param}.",
    "Determine that the complete total of {params} matches {param}.",
    "Reconfirm that the sum of {params} aligns with {param}.",
    "Substantiate that the total value of {params} is {param}.",
    "Verify the final sum of {params} equates to {param}."
    ]

abs_val_p1_minus_p2_equal_p_3=[
    "Check the absolute value of the difference between {param1} and {param2} equals {param3}.",
    "Confirm that the absolute difference between {param1} and {param2} is exactly {param3}.",
    "Check that the non-negative difference of {param1} and {param2} totals to {param3}.",
    "Verify that the positive distance between {param1} and {param2} measures {param3}.",
    "Ascertain that the absolute magnitude of the difference from {param1} to {param2} is {param3}.",
    "Make sure the absolute numerical gap between {param1} and {param2} is {param3}.",
    "Determine that the absolute value of subtracting {param2} from {param1} is {param3}.",
    "Validate the absolute distance from {param1} to {param2} as {param3}.",
    "Guarantee that the absolute separation between {param1} and {param2} equals {param3}.",
    "Affirm that the absolute variance of {param1} and {param2} is {param3}.",
    "Establish the absolute numerical difference of {param1} from {param2} as {param3}.",
    "Ensure the absolute deviation between {param1} and {param2} matches {param3}.",
    "Confirm that the absolute level of distinction between {param1} and {param2} is {param3}.",
    "Check the absolute numerical disparity of {param1} and {param2} against {param3}.",
    "Substantiate that the absolute measurement between {param1} and {param2} is {param3}.",
    "Reaffirm that the absolute value representing the difference of {param1} and {param2} is {param3}.",
    "Certify the absolute quantitative difference between {param1} and {param2} as {param3}.",
    "Make certain the absolute numerical divergence between {param1} and {param2} equals {param3}.",
    "Verify that the absolute value of the gap from {param1} to {param2} is {param3}.",
    "Ensure the absolute numerical contrast between {param1} and {param2} adheres to {param3}."
]


abs_val_param1_minus_param_2_diff_param3=[
    "Verify that the absolute value of the gap between {param1} and {param2} is not {param3}.",
    "Check that the absolute difference of {param1} and {param2} does not equal {param3}.",
    "Ascertain that the non-negative difference between {param1} and {param2} is not {param3}.",
    "Make sure the absolute disparity from {param1} to {param2} is unlike {param3}.",
    "Determine that the absolute value of {param1} minus {param2} is not equivalent to {param3}.",
    "Validate that the positive distance of {param1} from {param2} is not {param3}.",
    "Affirm that the absolute numerical difference between {param1} and {param2} isn't {param3}.",
    "Ensure that the absolute magnitude of difference from {param1} to {param2} isn't the same as {param3}.",
    "Guarantee that the absolute value of the discrepancy between {param1} and {param2} isn't {param3}.",
    "Establish that the absolute distance between {param1} and {param2} does not match {param3}.",
    "Confirm the absolute numerical gap between {param1} and {param2} is distinct from {param3}.",
    "Reaffirm that the absolute distance of {param1} and {param2} is not aligned with {param3}.",
    "Ensure the absolute deviation of {param1} from {param2} doesn't equal {param3}.",
    "Certify that the absolute numerical disparity between {param1} and {param2} isn't {param3}.",
    "Make certain the absolute value of the contrast between {param1} and {param2} isn't {param3}.",
    "Check that the absolute measurement of difference between {param1} and {param2} is not {param3}.",
    "Verify that the absolute separation from {param1} to {param2} is other than {param3}.",
    "Substantiate that the absolute separation of {param1} and {param2} doesn't mirror {param3}.",
    "Ensure the absolute numerical distance of {param1} from {param2} is not identical to {param3}.",
    "Confirm that the absolute value representing the differential of {param1} and {param2} diverges from {param3}."
]


In [7]:

maxlen=50


class MultiTaskModel(torch.nn.Module):
    def __init__(self, num_labels_classification, num_labels_token_classification):
        super(MultiTaskModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_labels_classification)
        self.token_classifier = torch.nn.Linear(self.bert.config.hidden_size, num_labels_token_classification)

    def forward(self, input_ids, attention_mask, task_name):
        outputs = self.bert(input_ids, attention_mask)
        sequence_output = outputs.last_hidden_state
        pooler_output = outputs.pooler_output

        if task_name == "sequence_classification":
            return self.classifier(pooler_output)
        elif task_name == "token_classification":
            return self.token_classifier(sequence_output)
        else:
            raise ValueError("Task name should be either 'sequence_classification' or 'token_classification'.")


num_labels_classification=17
num_labels_token_classification=11
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = MultiTaskModel(num_labels_classification, num_labels_token_classification)
model.load_state_dict(torch.load('/content/drive/MyDrive/model_dict_27_12_23.pth'))




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


<All keys matched successfully>

In [8]:
def eliminate_repetitions(s):
    # Regular expression pattern to find repeated sequences of 'X' followed by digits
    pattern = re.compile(r'(X\d+)\1+')
    # Replace repeated sequences with a single instance
    result = pattern.sub(r'\1', s)
    return result


In [9]:

def bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification):
    model.eval()
    constraint= {}
    #remove ponctuation:
    sentence = re.sub(r'[^\w\s]', '', sentence)
    #print(sentence)
    inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=maxlen, return_tensors="pt")
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]

    # get relation
    task_name="sequence_classification"
    outputs = model(ids, mask, task_name)
    logits = outputs[0]
    pred = torch.argmax(logits)

    constraint['rel']=pred.item()

    # get parameters
    task_name="token_classification"
    outputs = model(ids, mask, task_name)
    logits = outputs[0]
    active_logits = logits.view(-1, num_labels_token_classification) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [i for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)
    #print("wp_preds", wp_preds)
    word_level_predictions = []
    # we will remove cls, sep and pad
    for pair in wp_preds:
      if (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
        continue
      else:
        word_level_predictions.append(pair)

    #print("word_level_predictions", word_level_predictions)
    list_params= {pair[1] for pair in word_level_predictions if pair[1]!=0}
    #print("list params", list_params)


    for i in list_params:
      p= [pair[0] for pair in word_level_predictions if pair[1]==i]
      #print("ici p est ",p)
      param = ''.join(s.replace('##', '') for s in p)
      param=eliminate_repetitions(param)
      constraint[i]=param

    return constraint


In [10]:
class Relation:
  def __init__(self, operator , arity, directed,  parameter=-999):
    self.operator = operator

    self.arity = arity
    self.directed= directed
    self.parameter= parameter

In [11]:
class Constraint:
  def __init__(self, scope, rel):
    self.id=uuid.uuid4()
    self.scope = scope
    self.rel = rel

  def is_completely_instantiated(self, example):
      list_inst_vars=list(example.keys())
      #print(self.rel.operator, "on ",self.scope," list_inst_vars: ", list_inst_vars)
      l=[1 if v in list_inst_vars else 0 for v in self.scope ]
      if(sum(l)==len(self.scope)):
        return True
      else:
        return False

  def projection(self, example):
    keys= example.keys()
    proj= [(k, example[k]) for k in self.scope if k in keys]
    return(proj)

  def is_satisfied(self, example):
    proj=self.projection(example)
    #print(" ici la projecttion est...",proj)
    vals=[t[1] for t in proj]
    my_str=''
    val=self.rel.parameter
    if self.rel.operator in['<=', '!=','<','==','>','>=']:
      x=vals[0]
      y=vals[1]
      my_str="x"+self.rel.operator+"y"
    elif self.rel.operator=='sigma_diff':
      x=vals[0]
      y=vals[1]
      z=vals[2]
      my_str="x+y != z"
    elif self.rel.operator=='||=val':
      x=vals[0]
      y=vals[1]
      my_str="abs(x-y)==val"
    elif self.rel.operator=='||!=val':
      x=vals[0]
      y=vals[1]
      my_str="abs(x-y)!=val"
    elif self.rel.operator=='||>val':
      x=vals[0]
      y=vals[1]
      my_str="abs(x-y)>val"
    elif self.rel.operator.startswith('O_val_'):
      operator=self.rel.operator.split('O_val_')[1]
      x=vals[0]
      my_str="x"+ operator + "val"
      if operator == '=':
        my_str="x == val"
    elif self.rel.operator== '||xy_zt':
      x=vals[0]
      y=vals[1]
      z=vals[2]
      t=vals[3]
      my_str=" abs(x-y) == abs(z-t)"
    elif self.rel.operator== '||!=xy_zt':
      x=vals[0]
      y=vals[1]
      z=vals[2]
      t=vals[3]
      my_str=" abs(x-y) != abs(z-t)"
    elif self.rel.operator== 'Sum':
      my_str=[ f"vals[{i}]" for i in range(len(vals))]
      oper="+"
      my_str=oper.join(my_str)
      my_str=my_str+"=="+"val"
    elif self.rel.operator== 'all_diff':
      return len(tuple(vals))==len(set(vals))
      #print("vars = ", self.scope, " vas ", vals," ? ",val)

      #print("################## my str to be evaluated is ...",my_str, "and its value is ", eval(my_str))
      #sys.exit()

    if len(my_str) !=0:

      return eval(my_str)
    else:
      print(" unhandeled relation ??????????????????????????????????")
      sys.exit()
      return None

  def check_constraint(self, example):
    if self.is_completely_instantiated(example):
      if self.is_satisfied(example):
        return True
      else:
        return False
    else:
      return True



In [12]:
class Target_net():
  def __init__(self,constraints):
    self.constraints=constraints

  def ask(self,example):
    for c in self.constraints:
      check= c.check_constraint(example)
      if not check:
        #print("Inside ask---the following constraint is violated ", c.rel.operator, " on ", c.scope)
        return False,c
    return True, "_"

In [13]:
dict_constraints={
    0: {"constraint": "all_diff", "template":"templates_all_diff", "n_params":10},
    1: {"constraint": "<", "template":"templates_param1_strictely_lt_param2", "n_params":2},
    2: {"constraint": ">", "template":"templates_param1_strictely_gt_param2", "n_params":2},
    3: {"constraint": "||!=p", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3},
    4: {"constraint": "||xy_zt", "template":"abs_val_p1_minus_p2_equal_abs_val_p3_minus_p4", "n_params":4},
    5: {"constraint": "||=p", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    6: {"constraint": "sigma_diff", "template":"templates_sum_x1_x2_diff_x3", "n_params":3},
    7: {"constraint": "==", "template":"templates_equal_constraint", "n_params":2},
    8: {"constraint": "!=", "template":"templates_different_than", "n_params":2},
    9: {"constraint": "<=", "template":"templates_less_than", "n_params":2},
    10: {"constraint": ">=", "template":"templates_greater_than", "n_params":2},
    11: {"constraint": "||!=xy_zt", "template":"abs_val_p1_minus_p2_diff_abs_val_p3_minus_p4", "n_params":4},
    12: {"constraint": "Sum", "template":"sum_params_equal_param", "n_params":10},
    13: {"constraint": "O_val_=", "template":"templates_equal_constraint", "n_params":2},
    14: {"constraint": "O_val_!=", "template":"templates_different_than", "n_params":2},
    15: {"constraint": "||=val", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    16: {"constraint": "||!=val", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3}
}

In [14]:

def format_template(template, **kwargs):
    try:
        formatted_template = template.format(**kwargs)
        return formatted_template
    except KeyError as e:
        print(f"Error: Missing parameter {e}")
        return None

def fill_template_sum(template, param, *params):
    params = ', '.join(map(str, params))  # Corrected: No '*' before 'params' in join
    return template.format(params=params, param=param)  # Corrected: Named placeholders in format


dict_constraints=dict(dict_constraints)
opToIndex={ dict_constraints[i]["constraint"]: i for i in range(len(dict_constraints)) }
#print(opToIndex)

def generate_sythetic_sentence_for_constraint(c):
    i={opToIndex[c.rel.operator]}
    i=i.pop()
    #print("....i",i)
    params={}
    para=-1

    if i==0 :
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      selected_params = list( params.values())
      template = random.choice(templates_all_diff)
      param_template = ", ".join(selected_params)
      formatted_template=template.format(params=param_template)

    elif i==12:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      selected_params = list( params.values())
      para = c.rel.parameter
      template = random.choice(sum_params_equal_param)
      formatted_template = fill_template_sum(template, para, *selected_params)


    elif i in [13,14,15,16]:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      if c.rel.operator in["||!=val", "||=val"]:
        params["param3"]=str(c.rel.parameter)
      #print( " 2222 ||!=val ||=val ", c.scope, c.rel.operator)
      elif c.rel.operator in[ "O_val_=", "O_val_!="]:
        params["param2"]=str(c.rel.parameter)
      template = dict_constraints[i]["template"]
      template= random.choice(eval(template))
      formatted_template = format_template(template, **params)

    else:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      template = dict_constraints[i]["template"]
      template= random.choice(eval(template))
      formatted_template = format_template(template, **params)
      #print(formatted_template)

    return formatted_template,params,str(para)



In [15]:

def is_valid(s):

    # Regular expression pattern
    # ^ - start of the string
    # (X\d+|\d+) - matches either 'X' followed by one or more digits (\d+)
    # or just one or more digits
    # $ - end of the string
    pattern = re.compile(r'^(x\d+)$')
    # Check if the string matches the pattern
    return bool(pattern.match(s))

def compareConsTargVsConstBert(cT,cB, dict_constraints ):
  #first of all, we have to compare relations
  #print(" il s'agit d'une contrainte ", cT.rel.operator)
  relT=cT.rel.operator
  relB=dict_constraints[cB['rel']]["constraint"]
  #print(relT, relB)
  if relT != relB:
    return False

  # Then, we have to compare scopes
  para=-999
  scT= cT.scope
  scB= cB.values()
  scB= list(scB)
  scB=scB[1:]

  #print("nous allons comprer les scopes....", scB, " ...", scT)
  #print("0000 scB....",scB)
  if cT.rel.parameter !=-999:
    para= scB[-1]
    scB=scB[:-1]
  #print("scB....",scB)
  for s in scB:
    #print("....",s)
    if not is_valid(s):
      return False
  scB = [int(s[1:]) for s in scB]
  #print( " now scB",scB)

  #now we have to compare scopes and parameters
  #first case is when the constraint has no parameter and is not directed:
  if cT.rel.operator in ["all_diff", "==", "!=", "sigma_diff"]:
    scB=set(scB)
    scT=set(scT)
    #print(scB,scT)
    scB=set(scB)
    scT=set(scT)
    if scT != scB:
      #print("they are diff")
      return False

  if cT.rel.operator == "Sum":
    scB=set(scB)
    scT=set(scT)
    #print(scB,scT)
    scB=tuple(scB)
    scT=tuple(scT)
    #print(scB,scT)
    if scT != scB or int(para) != cT.rel.parameter:
      #print("they are diff")
      return False

  elif cT.rel.operator in [ "<", ">", "<=", " >= " ]:
    scB=tuple(scB)
    scT=tuple(scT)
    #print(scB,scT)
    if scT != scB:
      #print("they are diff")
      return False
  elif cT.rel.operator in [ "||xy_zt", "||!=xy_zt" ]:
    scB=tuple(scB)
    scT=tuple(scT)
    b0,b1,b2,b3=scB
    t0,t1,t2,t3=scT
    B0=set([b0,b1])
    B1=set([b2,b3])
    T0=set([t0,t1])
    T1=set([t2,t3])
    #print(T0,B0)
    if B0 != T0 or B1!= T1:
      #print("they are diff")
      return False
  elif cT.rel.operator in ["|p1-p2|!=p3", "|p1-p2|==p3"]:
    scB=tuple(scB)
    scT=tuple(scT)
    b0,b1,b2=scB
    t0,t1,t2=scT
    B0=set([b0,b1])
    B1= b3
    T0=set([t0,t1])
    T1= t3
    if B0 != T0 or B1!= T1:
      #print("they are diff")
      return False
  elif cT.rel.operator in ["||=val", "||!=val"]:
    scB=set(scB)
    scT=set(scT)
    print("scB....",scB)
    if scB != scT:
      return False
    B1= int(para)
    T1= cT.rel.parameter
    if B1!= T1:
      return False

  elif cT.rel.operator in ["O_val_=", "O_val_!="]:
    scB=tuple(scB)
    scT=tuple(scT)
    b0  =scB
    t0  =scT
    B0=b0
    B1= int(para)
    T0=t0
    T1= cT.rel.parameter
    #print("B0,T0",B0,T0," B1,T1",B1,T1)
    if B0 != T0 or B1!= T1:
      #print("they are diff")
      return False
  return True


In [16]:
###########################################################################
########################## SUDOKO PUZZLE GLOBAL ALLDIFF ###################
###########################################################################

"""
We need only 21 alldiff global constraints to model sudoku puzzle.
see: "Redundant Sudoku Rules", https://arxiv.org/pdf/1207.5926.pdf

"""

"""
row = dict({})
for i in range(9):
  row[i]=[i*9+j for j in range(9)]
  print(row[i])

print("")
col = dict({})
for j in range(9):
  col[j]=[i*9+j for i in range(9)]
  print(col[j])

k=0
block=dict({})
for row_block in range(3):
        for col_block in range(3):
            block[k] = [(row_block * 3 + row)*9+ (col_block * 3 + col) for row in range(3) for col in range(3)]
            print(block[k])
            k=k+1

r39= Relation('all_diff',9,True )
constraints=[]

for i in range(9):
  constraints.append(Constraint(row[i],r39))
  constraints.append(Constraint(col[i],r39))
  constraints.append(Constraint(block[i],r39))

target_net= Target_net(constraints)

nbr_vars=81
lower_bound_domain=1
upper_bound_domain=9
vars_indices= [i for i in range(nbr_vars)]
fixed_vars=[]
"""


'\nrow = dict({})\nfor i in range(9):\n  row[i]=[i*9+j for j in range(9)]\n  print(row[i])\n\nprint("")\ncol = dict({})\nfor j in range(9):\n  col[j]=[i*9+j for i in range(9)]\n  print(col[j])\n\nk=0\nblock=dict({})\nfor row_block in range(3):\n        for col_block in range(3):\n            block[k] = [(row_block * 3 + row)*9+ (col_block * 3 + col) for row in range(3) for col in range(3)]\n            print(block[k])\n            k=k+1\n\nr39= Relation(\'all_diff\',9,True )\nconstraints=[]\n\nfor i in range(9):\n  constraints.append(Constraint(row[i],r39))\n  constraints.append(Constraint(col[i],r39))\n  constraints.append(Constraint(block[i],r39))\n\ntarget_net= Target_net(constraints)\n\nnbr_vars=81\nlower_bound_domain=1\nupper_bound_domain=9\nvars_indices= [i for i in range(nbr_vars)]\nfixed_vars=[]\n'

In [17]:
###########################################################################
########################## SUDOKO PUZZLE BINARY DIFF ######################
###########################################################################


"""
Conjecture: No model with less than 648 binary constraints is Sudoku.

"...For each of the models in Missing(6) one can easily count the number of different
small constraints it represents: for the ones that are Sudoku, the highest count is 690,
and the lowest count is 648."

see: "Redundant Sudoku Rules", https://arxiv.org/pdf/1207.5926.pdf

"""

r6= Relation('!=',2, False )
row = dict({})
for i in range(9):
  row[i]=[i*9+j for j in range(9)]
  print(row[i])

print("")
col = dict({})
for j in range(9):
  col[j]=[i*9+j for i in range(9)]
  print(col[j])

k=0
block=dict({})
for row_block in range(3):
        for col_block in range(3):
            block[k] = [(row_block * 3 + row)*9+ (col_block * 3 + col) for row in range(3) for col in range(3)]
            print(block[k])
            k=k+1

constraints=[]


for k in range(9):
  print(row[k],f": row[{k}] ##############################")
  for i in row[k]:
    for j in row[k]:
      if j>i:
        constraints.append( Constraint([i,j],r6))
        print(i,j)

for k in range(9):
  print(col[k],f": col[{k}] ##############################")
  for i in col[k]:
    for j in col[k]:
      if j>i:
        constraints.append( Constraint([i,j],r6))
        print(i,j)

for k in range(9):
  print(block[k],f": block[{k}] ##############################")
  for i in block[k]:
    for j in block[k]:
      if j>i:
        constraints.append( Constraint([i,j],r6))
        print(i,j)


print("nombre contraintes....", len(constraints))
target_net= Target_net(constraints)

nbr_vars=81
lower_bound_domain=1
upper_bound_domain=9
vars_indices= [i for i in range(nbr_vars)]
fixed_vars=[]


[0, 1, 2, 3, 4, 5, 6, 7, 8]
[9, 10, 11, 12, 13, 14, 15, 16, 17]
[18, 19, 20, 21, 22, 23, 24, 25, 26]
[27, 28, 29, 30, 31, 32, 33, 34, 35]
[36, 37, 38, 39, 40, 41, 42, 43, 44]
[45, 46, 47, 48, 49, 50, 51, 52, 53]
[54, 55, 56, 57, 58, 59, 60, 61, 62]
[63, 64, 65, 66, 67, 68, 69, 70, 71]
[72, 73, 74, 75, 76, 77, 78, 79, 80]

[0, 9, 18, 27, 36, 45, 54, 63, 72]
[1, 10, 19, 28, 37, 46, 55, 64, 73]
[2, 11, 20, 29, 38, 47, 56, 65, 74]
[3, 12, 21, 30, 39, 48, 57, 66, 75]
[4, 13, 22, 31, 40, 49, 58, 67, 76]
[5, 14, 23, 32, 41, 50, 59, 68, 77]
[6, 15, 24, 33, 42, 51, 60, 69, 78]
[7, 16, 25, 34, 43, 52, 61, 70, 79]
[8, 17, 26, 35, 44, 53, 62, 71, 80]
[0, 1, 2, 9, 10, 11, 18, 19, 20]
[3, 4, 5, 12, 13, 14, 21, 22, 23]
[6, 7, 8, 15, 16, 17, 24, 25, 26]
[27, 28, 29, 36, 37, 38, 45, 46, 47]
[30, 31, 32, 39, 40, 41, 48, 49, 50]
[33, 34, 35, 42, 43, 44, 51, 52, 53]
[54, 55, 56, 63, 64, 65, 72, 73, 74]
[57, 58, 59, 66, 67, 68, 75, 76, 77]
[60, 61, 62, 69, 70, 71, 78, 79, 80]
[0, 1, 2, 3, 4, 5, 6, 7, 8] : 

In [18]:

######################################################
################## ZEBRA PROBLEM #####################
######################################################

"""
nbr_vars=25
lower_bound_domain=0
upper_bound_domain=4
vars_indices= [i for i in range(nbr_vars)]
fixed_vars=[]

idx={
    'Nor':0, 'En': 1, 'Sp' :2, 'Uk': 3, 'Jp': 4,
    'bl':5,'Red':6,'Gre':7, 'Yel':8, 'Iv':9,
    'Cof':10,'Mil':11,'te':12,'ora':13,'wat':14,
    'Fox':15,'dog':16,'snail':17,'hors':18,'zebr':19,
    'kool':20,'parl':21, 'old':22, 'luck':23,'chest':24
}

r1= Relation('O_val_=',1,False,0 )
r2= Relation('||=val',2,False,1 )
r3= Relation('O_val_=',1,False,2 )
r4= Relation('==',2, False )
r5= Relation('>',2, True )
r6= Relation('!=',2, False )

constraints=[]

c1=Constraint([idx['Nor']],r1) # =0 Norwegian lives in the first house on the left.
c2=Constraint([idx['Nor'],idx['bl']],r2) # ||=1 The Norwegian lives next to the blue house
c3= Constraint([idx['Mil']],r3)#=2 Milk is drunk in the middle house.
c4=Constraint([idx['En'], idx['Red'] ],r4) # == The Englishwoman lives in the red house.
c5=Constraint([idx['Cof'], idx['Gre'] ],r4) # == Coffee is drunk in the green house.
c6=Constraint([idx['kool'], idx['Yel'] ],r4) # == Kools are smoked in the yellow house.
c7=Constraint([idx['Iv'],idx['Gre']],r2) # ||=1  The green house is immediately to the right of the ivory house.
c8=Constraint([idx['Gre'],idx['Iv']],r5) # > The green house is immediately to the right of the ivory house.
c9=Constraint([idx['Sp'], idx['dog'] ],r4) #==  The Spaniard owns a dog.
c10=Constraint([idx['Uk'], idx['te'] ],r4) # == The Ukrainian drinks tea.
c11=Constraint([idx['Jp'], idx['parl'] ],r4) # == The Japanese smokes Parliament.
c12=Constraint([idx['old'], idx['snail'] ],r4) # == The Oldgold smoker owns the snail.
c13=Constraint([idx['luck'], idx['ora'] ],r4) # == The Lucky Strike smoker drinks orange juice.
c14=Constraint([idx['chest'],idx['Fox']],r2) # ||=1 Chesterfield smoker lives next to the fox owner.
c15=Constraint([idx['Yel'],idx['hors']],r2) # ||=1

c16=Constraint([0,1,2,3,4],r39) # all diff
c17=Constraint([5,6,7,8,9],r39) # all diff
c18=Constraint([10,11,12,13,14],r39) # all diff
c19=Constraint([15,16,17,18,19],r39) # all diff
c20=Constraint([20,21,22,23,24],r39) # all diff



constraints=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20]

target_net= Target_net(constraints)

#first_clique=[0,1,2,3,4]
#sec_clique=[5,6,7,8,9]
#th_clique=[10,11,12,13,14]
#four_clique=[15,16,17,18,19]
#fif_clique=[20,21,22,23,24]

#for i in first_clique:
#  for j in first_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in sec_clique:
#  for j in sec_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in th_clique:
#  for j in th_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in four_clique:
#  for j in four_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in fif_clique:
#  for j in fif_clique:
#    if j>i:
#      constraints.append(Constraint([i,j],r6))

"""

"\nnbr_vars=25\nlower_bound_domain=0\nupper_bound_domain=4\nvars_indices= [i for i in range(nbr_vars)]\nfixed_vars=[]\n\nidx={\n    'Nor':0, 'En': 1, 'Sp' :2, 'Uk': 3, 'Jp': 4,\n    'bl':5,'Red':6,'Gre':7, 'Yel':8, 'Iv':9,\n    'Cof':10,'Mil':11,'te':12,'ora':13,'wat':14,\n    'Fox':15,'dog':16,'snail':17,'hors':18,'zebr':19,\n    'kool':20,'parl':21, 'old':22, 'luck':23,'chest':24\n}\n\nr1= Relation('O_val_=',1,False,0 )\nr2= Relation('||=val',2,False,1 )\nr3= Relation('O_val_=',1,False,2 )\nr4= Relation('==',2, False )\nr5= Relation('>',2, True )\nr6= Relation('!=',2, False )\n\nconstraints=[]\n\nc1=Constraint([idx['Nor']],r1) # =0 Norwegian lives in the first house on the left.\nc2=Constraint([idx['Nor'],idx['bl']],r2) # ||=1 The Norwegian lives next to the blue house\nc3= Constraint([idx['Mil']],r3)#=2 Milk is drunk in the middle house.\nc4=Constraint([idx['En'], idx['Red'] ],r4) # == The Englishwoman lives in the red house.\nc5=Constraint([idx['Cof'], idx['Gre'] ],r4) # == Coffee 

In [19]:
######################################################
################## PURDEY PROBLEM #####################
######################################################

"""
nbr_vars=12
lower_bound_domain=0
upper_bound_domain=3
vars_indices= [i for i in range(nbr_vars)]
fixed_vars=[]

idx={
    'Boyds':0, 'Garveys': 1, 'Logans' :2, 'Navarros': 3,
    'Flour': 4,'Kerosene':5,'Muslin':6,'Sugar':7,
    'Cash':8, 'Credit':9,'Traded_Ham':10,'Traded_Peas':11,
    }

# 1- The Boyds were new in town, and this was their first visit to the store
# 2- The family (which wasn't the Logan's) that traded the bushel of peas didn't buy the kerosene. ==> Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene
# 3- The Boyds and the Garveys bought the kerosene and the muslin in some order
# 4- One family traded a cured ham for a large sack of flour ==> Traded_Ham = flour
# 5- Purdey only extended credit to regular customers, such that the family that bought the muslin on credit

# constraints:
# 2:  Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene
# 3: (Boyds = kerosene and the Garveys = muslin) [[[[[ or (Boyds = muslin  and the Garveys = kerosene ) ]]]]] ?
# 4:  Traded_Ham = flour
# 5: Muslin = credit
# 1 and 5: Boyds !=credit




r1= Relation('<=', 2, True)
r4= Relation('==',2, False )
r6= Relation('!=',2, False )
r39= Relation('all_diff',10,True )


constraints=[]


#c1=Constraint([0,5],r4)
#c2=Constraint([0,8],r4)
#c3=Constraint([1,6],r4)

#c4=Constraint([1,9],r4)
#c5=Constraint([2,4],r4)

#c6=Constraint([2,10],r4)
#c7=Constraint([3,7],r4)

#c8=Constraint([3,11],r4)
#c9=Constraint([5,8],r4)

#c10=Constraint([6,9],r4)
#c11=Constraint([4,10],r4)
#c12=Constraint([7,11],r4)


c1=Constraint([2,11],r6) #  Logans !=Traded_Peas
c2=Constraint([11,5],r6) #  Traded_Peas !=Kerosene
c3=Constraint([4,10],r4) #Traded_Ham = flour
c4=Constraint([6,9],r4) # Muslin = credit
c5=Constraint([0,9],r6) #   Boyds !=credit
c6=Constraint([0,5],r4) #Boyds = kerosene
c7=Constraint([1,6],r4) #Garveys = muslin
c8=Constraint([0,1],r1) #Boyds < Garveys
c9=Constraint([1,2],r1) #Garveys <= Logans
c10=Constraint([2,3],r1) #Logans <= Navarros


li=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10] # adding constraints for breaking symetries
#li=[c1,c2,c3,c4,c5,c6,c7]# without breaking symetries
constraints.extend(li)

first_clique=[0,1,2,3]
sec_clique=[4,5,6,7]
th_clique=[8,9,10,11]

constraints.append( Constraint(first_clique,r39))
constraints.append( Constraint(sec_clique,r39))
constraints.append( Constraint(th_clique,r39))


print("le nombre des contraintes est ", len(constraints))

target_net= Target_net(constraints)

"""


'\nnbr_vars=12\nlower_bound_domain=0\nupper_bound_domain=3\nvars_indices= [i for i in range(nbr_vars)]\nfixed_vars=[]\n\nidx={\n    \'Boyds\':0, \'Garveys\': 1, \'Logans\' :2, \'Navarros\': 3,\n    \'Flour\': 4,\'Kerosene\':5,\'Muslin\':6,\'Sugar\':7,\n    \'Cash\':8, \'Credit\':9,\'Traded_Ham\':10,\'Traded_Peas\':11,\n    }\n\n# 1- The Boyds were new in town, and this was their first visit to the store\n# 2- The family (which wasn\'t the Logan\'s) that traded the bushel of peas didn\'t buy the kerosene. ==> Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene\n# 3- The Boyds and the Garveys bought the kerosene and the muslin in some order\n# 4- One family traded a cured ham for a large sack of flour ==> Traded_Ham = flour\n# 5- Purdey only extended credit to regular customers, such that the family that bought the muslin on credit\n\n# constraints:\n# 2:  Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene\n# 3: (Boyds = kerosene and the Garveys = muslin) [[[[[ or (Boyds = muslin  and the

In [20]:
######################################################
################## KAKURU PROBLEM #####################
######################################################
"""
grid = [
    [0, 0, 0, 0, 0, -1],
    [0, 0, 0, 0, 0, 0],
    [0, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [-1, 0, 0, 0, 0, 0]
]
"""

"""
Kakuro1
horizontal_clues = [
    {34: [(0, 0), (0, 1), (0, 2),(0,3),(0,4)]}, #34
    {39: [(1, 0), (1, 1), (1, 2),(1,3),(1,4),(1,5)]},
    {4:  [(2, 0), (2, 1)]},#4
    {8:  [(2, 4), (2, 5)]},
    {7:  [(3, 0), (3, 1)]}, #7
    {9:  [(3, 4), (3, 5)]},#9
    {35: [(4, 0), (4, 1), (4, 2),(4,3),(4,4),(4,5)]},
    {15: [ (5, 1), (5, 2),(5,3),(5,4),(5,5)]}# 15
]



vertical_clues = [
    {15: [(0, 0), (1, 0), (2, 0), (3, 0),(4,0)]},
    {21: [(0, 1), (1, 1), (2, 1), (3, 1),(4,1),(5,1)]},
    {13: [(0, 2), (1, 2)]},
    {13: [(4,2),(5,2)]},
    {15: [(0, 3), (1, 3)]},
    {9:  [(4,3),(5,3)]},
    {30: [(0, 4), (1, 4), (2, 4), (3, 4),(4,4),(5,4)]},
    {35: [(1, 5), (2, 5), (3, 5),(4,5),(5,5)]}
]
"""

"""
Kakuro93
horizontal_clues = [
    {34: [(0, 0), (0, 1), (0, 2),(0,3),(0,4)]}, #34
    {33: [(1, 0), (1, 1), (1, 2),(1,3),(1,4),(1,5)]},
    {4:  [(2, 0), (2, 1)]},#4
    {8:  [(2, 4), (2, 5)]},
    {7:  [(3, 0), (3, 1)]}, #7
    {9:  [(3, 4), (3, 5)]},#9
    {35: [(4, 0), (4, 1), (4, 2),(4,3),(4,4),(4,5)]},
    {19: [ (5, 1), (5, 2),(5,3),(5,4),(5,5)]}# 15
]



vertical_clues = [
    {15: [(0, 0), (1, 0), (2, 0), (3, 0),(4,0)]},
    {21: [(0, 1), (1, 1), (2, 1), (3, 1),(4,1),(5,1)]},
    {13: [(0, 2), (1, 2)]},
    {13: [(4,2),(5,2)]},
    {15: [(0, 3), (1, 3)]},
    {9:  [(4,3),(5,3)]},
    {28: [(0, 4), (1, 4), (2, 4), (3, 4),(4,4),(5,4)]},
    {35: [(1, 5), (2, 5), (3, 5),(4,5),(5,5)]}
]
"""

"""
n=len(grid)
vars_indices=[i for i in range(n*n)]
nbr_vars= len(vars_indices)
fixed_vars=[]

lower_bound_domain=1
upper_bound_domain=9


r39= Relation('all_diff',10,True )


constraints=[]

for i in range(n):
  for j in range(n):
    if grid[i][j]==-1:
      fixed_vars.append(i*n+j)

print("printing fixed_vars",fixed_vars)


print("printing horizontal clues")
for h_c in horizontal_clues:
  clue= list(h_c.keys())[0]
  l= list(h_c.values())[0]
  list_vars = [t[0]*n+t[1] for t in  l]
  print(clue,  list_vars)
  constraints.append(Constraint(list_vars, Relation('Sum',10,False,clue)))
  constraints.append(Constraint(list_vars, r39))

print("printing vertical clues")
for v_c in vertical_clues:
  clue= list(v_c.keys())[0]
  l= list(v_c.values())[0]
  list_vars = [t[0]*n+t[1] for t in  l]
  print(clue,  list_vars)
  constraints.append(Constraint(list_vars, Relation('Sum',10,False,clue)))
  constraints.append(Constraint(list_vars, r39))


print("le nombre des contraintes est ", len(constraints))
target_net= Target_net(constraints)
"""

'\nn=len(grid)\nvars_indices=[i for i in range(n*n)]\nnbr_vars= len(vars_indices)\nfixed_vars=[]\n\nlower_bound_domain=1\nupper_bound_domain=9\n\n\nr39= Relation(\'all_diff\',10,True )\n\n\nconstraints=[]\n\nfor i in range(n):\n  for j in range(n):\n    if grid[i][j]==-1:\n      fixed_vars.append(i*n+j)\n\nprint("printing fixed_vars",fixed_vars)\n\n\nprint("printing horizontal clues")\nfor h_c in horizontal_clues:\n  clue= list(h_c.keys())[0]\n  l= list(h_c.values())[0]\n  list_vars = [t[0]*n+t[1] for t in  l]\n  print(clue,  list_vars)\n  constraints.append(Constraint(list_vars, Relation(\'Sum\',10,False,clue)))\n  constraints.append(Constraint(list_vars, r39))\n\nprint("printing vertical clues")\nfor v_c in vertical_clues:\n  clue= list(v_c.keys())[0]\n  l= list(v_c.values())[0]\n  list_vars = [t[0]*n+t[1] for t in  l]\n  print(clue,  list_vars)\n  constraints.append(Constraint(list_vars, Relation(\'Sum\',10,False,clue)))\n  constraints.append(Constraint(list_vars, r39))\n\n\nprint("

In [21]:
def add_constraint_to_cpModel(constraint,model,vars, abs_vars):
    c= constraint
    n=len(vars)
    i=j=u=w=-1
    val=c.rel.parameter
    if(c.rel.arity==1):
      i=c.scope[0]
    elif(c.rel.arity==2):
      i,j=c.scope
    elif(c.rel.arity==3):
      i,j,u=c.scope
    elif(c.rel.arity==4):
      i,j,u,w=c.scope
    if "<=" == c.rel.operator:
      model.Add(vars[i] <= vars[j])
    elif ">=" == c.rel.operator:
      model.Add(vars[i] >= vars[j])
    elif "!=" == c.rel.operator:
      model.Add(vars[i] != vars[j])
    elif "sigma_diff" == c.rel.operator:
      model.Add(vars[i] + vars[j] != vars[u])
    elif "<" == c.rel.operator:
      model.Add(vars[i] < vars[j])
    elif ">" == c.rel.operator:
      model.Add(vars[i] > vars[j])
    elif "==" == c.rel.operator:
      model.Add(vars[i] == vars[j])

    elif "||=val" == c.rel.operator:
      model.AddAbsEquality(val, vars[i] - vars[j])
    elif "||!=val" == c.rel.operator:
      #print(" len of abs_vars:  ", len(abs_vars), ",j,n, i*n+j", i, j, n, i*n+j )
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] != val)
    elif "||>val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] > val)
    elif "||<val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] < val)
    elif "||>=val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] >= val)
    elif "||<=val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] <= val)

    elif "O_val_<=" == c.rel.operator:
      model.Add(vars[i]<=val)
    elif "O_val_>=" == c.rel.operator:
      model.Add(vars[i]>=val)
    elif "O_val_<" == c.rel.operator:
      model.Add(vars[i]<val)
    elif "O_val_>" == c.rel.operator:
      model.Add(vars[i]>val)
    elif "O_val_!=" == c.rel.operator:
      model.Add(vars[i]!=val)
    elif "O_val_=" == c.rel.operator:
      model.Add(vars[i]==val)

    elif "||xy_zt" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.AddAbsEquality(abs_vars[u*n+w], vars[u] - vars[w])
      model.Add(abs_vars[i*n+j] == abs_vars[u*n+w])

    elif "||!=xy_zt" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.AddAbsEquality(abs_vars[u*n+w], vars[u] - vars[w])
      model.Add(abs_vars[i*n+j] != abs_vars[u*n+w])

    elif "Sum" == c.rel.operator:
      involved_vars= [vars[i] for i in c.scope]
      #print(" involved vars: ", [v.Name() for v in involved_vars])
      #sys.exit()
      model.Add(sum(involved_vars) == val)
    elif "all_diff" == c.rel.operator:
      involved_vars= [vars[i] for i in c.scope]
      model.AddAllDifferent(involved_vars)

    else:
      print(f"Unsupported constraint: {c.rel.operator}")
      sys.exit()



def generateExample(learned_tuples, learned_constraints, fixed_vars ):


    model = cp_model.CpModel()

    # Creates the variables.

    vars=[]
    for i in range(nbr_vars):
      vars.append(model.NewIntVar(lower_bound_domain, upper_bound_domain, "x_"+str(i)))


    abs_vars= [ model.NewIntVar(0, 2*upper_bound_domain, "abs_"+str(k)) for k in range(nbr_vars * nbr_vars) ]

    for i in range(nbr_vars):
      for j in range(nbr_vars):
        model.Add(abs_vars[i*nbr_vars+j] == abs_vars[j*nbr_vars+i])
        model.AddAbsEquality(abs_vars[i*nbr_vars+j], vars[i] - vars[j])

    # Creates the constraints.
    for c in learned_constraints:
          add_constraint_to_cpModel(c,model,vars,abs_vars)

    for i in fixed_vars:
      model.Add(vars[i] == 1)



    # Creates the constraints.
    for e in learned_tuples:
      forbidden_tuple = tuple(list(e.values()))
      #print(forbidden_tuple)
      inv_vars= [vars[i] for i in e.keys()]
      #print(inv_vars)
      model.AddForbiddenAssignments(inv_vars, [forbidden_tuple])


    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()

    solver.parameters.randomize_search = True
    solver.parameters.random_seed = 42


    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        #print("\n".join([f"{var.Name()} = {solver.Value(var)}" for var in vars]))
        proj= {i: solver.Value(vars[i])for i in range(len(vars))}
        return proj
    else:
        #print("No solution found.")
        return({})

    proj= [solver.Value(vars[x])for x in range(nbr_vars)]
    return proj

class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0
        self.__variables = variables

    def on_solution_callback(self):
        self.__solution_count += 1
        print(f'Solution {self.__solution_count}:')
        for v in self.__variables:
            print(f'  {v.Name()} = {self.Value(v)}')

    def solution_count(self):
        return self.__solution_count

def solveTargetNetwork(constraints, fixed_vars ):

    print(fixed_vars)

    model = cp_model.CpModel()

    # Creates the variables.

    vars=[]
    for i in range(nbr_vars):
      vars.append(model.NewIntVar(lower_bound_domain, upper_bound_domain, "x_"+str(i)))


    abs_vars= [ model.NewIntVar(0, 2*upper_bound_domain, "abs_"+str(k)) for k in range(nbr_vars * nbr_vars) ]


    for i in range(nbr_vars):
      for j in range(nbr_vars):
        model.Add(abs_vars[i*nbr_vars+j] == abs_vars[j*nbr_vars+i])
        model.AddAbsEquality(abs_vars[i*nbr_vars+j], vars[i] - vars[j])

    # Creates the constraints.
    for c in constraints:
      print(" adding ",c.rel.operator, " on ", c.scope)
      add_constraint_to_cpModel(c,model,vars,abs_vars)

    for i in fixed_vars:
      model.Add(vars[i] == 1)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    solution_printer = SolutionPrinter(vars)
    status = solver.SearchForAllSolutions(model, solution_printer)




In [22]:
#solveTargetNetwork(constraints, fixed_vars)

In [23]:
##### some usefull functions.
def exps_union(e1,e2):
  v_1= list(e1.keys())
  #print("#",v_1)
  v_2= list(e2.keys())
  v= sorted(set(v_1+v_2))
  merged_e = {k: e1[k] for k in v if k in v_1}
  merged_e.update({k: e2[k] for k in v if k not in v_1})
  return merged_e

def split_exple(e):
  li= list(e.items())
  size=len(li)
  size_left = math.ceil(size/2)
  left={}
  right={}
  for k in range(size_left):
    key, value = li[k]
    left[key]=value
  for k in range(size_left,size):
    key, value = li[k]
    right[key]=value
  return left,right

def difference(l1,l2):
  difference = list(set(l1) - set(l2))
  return difference


In [24]:
def findTuple(R,Y, Learned_constraints):
  global cpt
  e_R=R
  cpt =cpt+1
  #####################################################################
  answer,cT= target_net.ask(e_R)
  #### cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
  file.write(f"{cpt},,,1,,{answer}\n")
  ###########################################################################

  if not answer:
      print(" nous sommes ds find tuple, et c'est un exemple negatif. Pouvez donner une seule raison? ")
      sentence,_,_ = generate_sythetic_sentence_for_constraint(cT)
      print(sentence)
      cB= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)
      cpt=cpt+1
      ###############################################################################
      comp=compareConsTargVsConstBert(cT,cB,dict_constraints )
      print(" bert a réussi ?", comp)
      #### cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
      file.write(f"{cpt},,1,,,{comp}\n")
      #############################################################################
      if comp:
        Learned_constraints.append(cT)
        return "Exit"
      else:
        return {}

  if len(Y)==1:
    return Y


  Y1,Y2 = split_exple(Y)
  R_union_Y1= exps_union(R,Y1)
  S1=findTuple(R_union_Y1,Y2, Learned_constraints)
  if S1=="Exit":
    return "Exit"
  R_union_S1=exps_union(R,S1)
  S2={}
  S2=findTuple(R_union_S1,Y1, Learned_constraints)
  if S2=="Exit":
    return "Exit"
  return exps_union(S1,S2)

In [25]:

def LLMAcq():
  global cpt
  cutoff=0
  L=[]
  sols=[]
  L_union_sols=[]
  Learned_constraints=[]
  convergence = False
  while not convergence:
    print(" cpt ",cpt)
    if cutoff>10:
      return L, Learned_constraints

    L_union_sols=L+sols
    e=generateExample(L_union_sols, Learned_constraints,fixed_vars)
    if(len(e)==0):
      print("convergence......")
      return L, Learned_constraints
    cpt=cpt+1
    ########################################################################
    answer,cT = target_net.ask(e)
    # cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
    file.write(f"{cpt},,,,1,{answer}\n")
    #########################################################################
    if answer:
      sols.append(e)
      cutoff=cutoff+1
      print(" sol ", e)
    else:
      cutoff=0
      sentence,_,_ = generate_sythetic_sentence_for_constraint(cT)
      print("c'est un exemple negatif, pouvez expliquer pourquoi?")
      print(sentence)
      cB= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)
      cpt=cpt+1
      ##########################################################################
      comp=compareConsTargVsConstBert(cT,cB,dict_constraints )
      print(" bert a réussi ? ", comp)
      ####### cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
      file.write(f"{cpt},1,,,,{comp}\n")
      ##########################################################################
      if comp:
        Learned_constraints.append(cT)
      else:
        forbiddenTuple=findTuple({},e, Learned_constraints)
        #print(" forbiddenTuple .....", forbiddenTuple)
        if forbiddenTuple != "Exit":
          L.append(forbiddenTuple)


In [26]:
file_path = "/content/drive/MyDrive/results_queries_zebra_binary220424.txt"
file_path2 = "/content/drive/MyDrive/results_cpus_tuples_zebra_binary220424.txt"

with open(file_path, 'w') as file, open(file_path2, 'w') as file2:
    # Write the string to the file
    file.write("cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer\n")
    file2.write("#tuples,#constraints,#cpu\n")
    for k in range(10):
      cpt=0
      start_cpu = time.process_time()
      learned_Tuples, learned_c= LLMAcq()
      end_cpu = time.process_time()
      cpu = end_cpu - start_cpu
      print("number of learned tuples: ", len(learned_Tuples))
      print("number of learned constraints: ", len(learned_c))
      file2.write(f"{len(learned_Tuples)},{len(learned_c)},{cpu}\n")
      for t in learned_Tuples:
        print(f"{t}\n")
      for c in learned_c:
        print(f"{c.rel.operator} on {c.scope}\n")



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

!= on [64, 73]

!= on [40, 44]

!= on [73, 79]

!= on [75, 80]

!= on [37, 73]

!= on [19, 46]

!= on [19, 73]

!= on [73, 80]

!= on [19, 64]

!= on [37, 64]

!= on [28, 46]

!= on [28, 55]

!= on [37, 55]

!= on [46, 73]

!= on [10, 17]

!= on [46, 64]

!= on [2, 8]

!= on [55, 64]

!= on [0, 8]

!= on [2, 11]

!= on [2, 47]

!= on [2, 65]

!= on [2, 74]

!= on [11, 47]

!= on [12, 21]

!= on [11, 56]

!= on [11, 65]

!= on [11, 74]

!= on [20, 47]

!= on [38, 56]

!= on [29, 56]

!= on [38, 65]

!= on [29, 65]

!= on [29, 38]

!= on [20, 38]

!= on [38, 47]

!= on [3, 21]

!= on [47, 56]

!= on [47, 65]

!= on [29, 74]

!= on [20, 74]

!= on [47, 74]

!= on [74, 80]

!= on [64, 71]

!= on [2, 29]

!= on [27, 63]

!= on [55, 73]

!= on [3, 39]

!= on [56, 74]

!= on [65, 74]

!= on [56, 65]

!= on [46, 53]

!= on [12, 57]

!= on [3, 57]

!= on [12, 66]

!= on [3, 66]

!= on [12, 75]

!= on [3, 75]

!= on [